In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
raw_data = pd.read_excel('Jovie- Test.xlsx', sheet_name='Raw Data')
correct_data = pd.read_excel('Jovie- Test.xlsx', sheet_name='Correct Values')

In [4]:
correct_data.head()

,PREFERRED CAMPUS,PREFERRED COURSE
0,PHINMA St. Jude College - Quezon City,AB - Communication
1,PHINMA St. Jude College - Manila,AB - English Language
2,NaN,AB - Political Science
3,NaN,AB - Psychology
4,NaN,AB-English Language


In [5]:
correct_data.rename(columns = {'PREFERRED CAMPUS':'CORRECT CAMPUS', 'PREFERRED COURSE': 'CORRECT COURSE'}, inplace = 1)

In [6]:
correct_data

,CORRECT CAMPUS,CORRECT COURSE
0,PHINMA St. Jude College - Quezon City,AB - Communication
1,PHINMA St. Jude College - Manila,AB - English Language
2,NaN,AB - Political Science
3,NaN,AB - Psychology
4,NaN,AB-English Language
5,NaN,AB-Political Science
6,NaN,Bachelor in Early Childhood Education
7,NaN,BEED
8,NaN,BS Accountancy
9,NaN,BS Accounting Information System


In [7]:
raw_data.isnull().any()

PREFERRED CAMPUS    True
PREFERRED COURSE    True
dtype: bool

In [8]:
correct_data.isnull().any()

CORRECT CAMPUS     True
CORRECT COURSE    False
dtype: bool

In [9]:
raw_data

,PREFERRED CAMPUS,PREFERRED COURSE
0,PHINMA St. Jude College - Quezon City,BS Criminology
1,PHINMA St. Jude College - Quezon City,BS Accountancy
2,PHINMA St. Jude College - Quezon City,OTHERS
3,PHINMA St. Jude College - Quezon City,BS Hospitality Management
4,PHINMA St. Jude College - Quezon City,BS Hospitality Management
...,...,...
67102,PHINMA St. Jude College - Manila,BS Nursing
67103,PHINMA St. Jude College - Manila,BS Tourism Management
67104,PHINMA St. Jude College - Manila,BS Psychology
67105,PHINMA St. Jude College - Manila,BSED - English


In [10]:
correct_data['CAMPUS BRANCH'] = raw_data['PREFERRED CAMPUS'].str.split(pat = '-').str.get(1)

In [11]:
correct_data['CORRECT CAMPUS'] = correct_data['CORRECT CAMPUS'].fillna('N/A')

In [12]:
valid_campuses = {}

# Iterate over the rows of the DataFrame
for index, row in correct_data.iterrows():
    full_campus = row['CORRECT CAMPUS']
    if '-' in full_campus:
        # Split the string at the hyphen and get the part after the hyphen
        campus_part = full_campus.split('-')[1].strip()
        if full_campus not in valid_campuses:
            valid_campuses[full_campus] = campus_part

In [13]:
raw_data.dropna(subset=['PREFERRED CAMPUS'], inplace=True)

In [14]:
# Function to find the key based on the value
def find_key_by_value(value, dictionary):
    for key, val in dictionary.items():
        if val in value:
            return key
    return None

# Create a new column in raw_data based on the condition
raw_data['MATCHED CAMPUS'] = raw_data['PREFERRED CAMPUS'].apply(lambda x: find_key_by_value(x, valid_campuses))

In [15]:
raw_data[~raw_data['PREFERRED CAMPUS'].isin(['PHINMA St. Jude College - Quezon City',
    'PHINMA St. Jude College - Manila'])]

,PREFERRED CAMPUS,PREFERRED COURSE,MATCHED CAMPUS
41049,PHINMA SJC,OTHERS,None
41050,PHINMA SJC,OTHERS,None
41051,PHINMA SJC,OTHERS,None
41052,PHINMA SJC,OTHERS,None
41053,PHINMA SJC,OTHERS,None
...,...,...,...
62881,PHINMA SJC,BS Psychology,None
62882,PHINMA SJC,OTHERS,None
62883,PHINMA SJC,BS Nursing,None
62884,PHINMA SJC,BS Criminology,None


In [16]:
processed_data = raw_data.dropna(subset='MATCHED CAMPUS')

In [17]:
processed_data

,PREFERRED CAMPUS,PREFERRED COURSE,MATCHED CAMPUS
0,PHINMA St. Jude College - Quezon City,BS Criminology,PHINMA St. Jude College - Quezon City
1,PHINMA St. Jude College - Quezon City,BS Accountancy,PHINMA St. Jude College - Quezon City
2,PHINMA St. Jude College - Quezon City,OTHERS,PHINMA St. Jude College - Quezon City
3,PHINMA St. Jude College - Quezon City,BS Hospitality Management,PHINMA St. Jude College - Quezon City
4,PHINMA St. Jude College - Quezon City,BS Hospitality Management,PHINMA St. Jude College - Quezon City
...,...,...,...
67102,PHINMA St. Jude College - Manila,BS Nursing,PHINMA St. Jude College - Manila
67103,PHINMA St. Jude College - Manila,BS Tourism Management,PHINMA St. Jude College - Manila
67104,PHINMA St. Jude College - Manila,BS Psychology,PHINMA St. Jude College - Manila
67105,PHINMA St. Jude College - Manila,BSED - English,PHINMA St. Jude College - Manila


In [18]:
processed_data = processed_data.drop_duplicates(subset=['PREFERRED CAMPUS', 'PREFERRED COURSE'])

In [19]:
processed_data

,PREFERRED CAMPUS,PREFERRED COURSE,MATCHED CAMPUS
0,PHINMA St. Jude College - Quezon City,BS Criminology,PHINMA St. Jude College - Quezon City
1,PHINMA St. Jude College - Quezon City,BS Accountancy,PHINMA St. Jude College - Quezon City
2,PHINMA St. Jude College - Quezon City,OTHERS,PHINMA St. Jude College - Quezon City
3,PHINMA St. Jude College - Quezon City,BS Hospitality Management,PHINMA St. Jude College - Quezon City
5,PHINMA St. Jude College - Quezon City,NaN,PHINMA St. Jude College - Quezon City
...,...,...,...
35119,PHINMA St. Jude College - Manila,TVL-ICT,PHINMA St. Jude College - Manila
37324,PHINMA St. Jude College - Manila,BSED - Math,PHINMA St. Jude College - Manila
37664,PHINMA St. Jude College - Manila,BSED - Science,PHINMA St. Jude College - Manila
40180,PHINMA St. Jude College - Manila,BSBA - Marketing Management,PHINMA St. Jude College - Manila


In [20]:
processed_data.dropna(subset='PREFERRED COURSE')

,PREFERRED CAMPUS,PREFERRED COURSE,MATCHED CAMPUS
0,PHINMA St. Jude College - Quezon City,BS Criminology,PHINMA St. Jude College - Quezon City
1,PHINMA St. Jude College - Quezon City,BS Accountancy,PHINMA St. Jude College - Quezon City
2,PHINMA St. Jude College - Quezon City,OTHERS,PHINMA St. Jude College - Quezon City
3,PHINMA St. Jude College - Quezon City,BS Hospitality Management,PHINMA St. Jude College - Quezon City
28,PHINMA St. Jude College - Quezon City,BS Information Technology,PHINMA St. Jude College - Quezon City
...,...,...,...
35119,PHINMA St. Jude College - Manila,TVL-ICT,PHINMA St. Jude College - Manila
37324,PHINMA St. Jude College - Manila,BSED - Math,PHINMA St. Jude College - Manila
37664,PHINMA St. Jude College - Manila,BSED - Science,PHINMA St. Jude College - Manila
40180,PHINMA St. Jude College - Manila,BSBA - Marketing Management,PHINMA St. Jude College - Manila


In [21]:
final_data = processed_data[['MATCHED CAMPUS', 'PREFERRED COURSE']]

In [22]:
final_data = final_data.dropna(subset=['PREFERRED COURSE'])

In [23]:
# List of correct courses
correct_courses = correct_data['CORRECT COURSE'].tolist()

In [24]:
correct_courses

['AB - Communication',
 'AB - English Language',
 'AB - Political Science',
 'AB - Psychology',
 'AB-English Language',
 'AB-Political Science',
 'Bachelor in Early Childhood Education',
 'BEED',
 'BS Accountancy',
 'BS Accounting Information System',
 'BS Accounting Technology',
 'BS Architechture',
 'BS Architecture',
 'BS Civil Engineering',
 'BS Computer Engineering',
 'BS Computer Science',
 'BS Criminology',
 'BS Early Childhood Education',
 'BS Electrical Engineering',
 'BS Electronics Engineering',
 'BS Entrepreneurship',
 'BS Hospitality Management',
 'BS Information Technology',
 'BS Management Accounting',
 'BS Marine Engineering',
 'BS Mechanical Engineering',
 'BS Medical Laboratory Science',
 'BS Medical Technology',
 'BS Nursing',
 'BS Pharmacy',
 'BS Physical Therapy',
 'BS Psychology',
 'BS Radiologic Technology',
 'BS Respiratory Therapy',
 'BS Tourism Management',
 'BSBA - Financial Management',
 'BSBA - Financial Marketing',
 'BSBA - Human Resource Management',
 'BS

In [30]:
# Find best match from correct_courses
def find_best_match(preferred_course):
    preferred_course = preferred_course.strip()
    
    # Fuzzy matching with different ratios
    match_ratio, score_ratio = process.extractOne(preferred_course, correct_courses, scorer=fuzz.ratio)
    match_partial, score_partial = process.extractOne(preferred_course, correct_courses, scorer=fuzz.partial_ratio)
    match_token_sort, score_token_sort = process.extractOne(preferred_course, correct_courses, scorer=fuzz.token_sort_ratio)
    
    # Combine scores
    combined_scores = [
        (match_ratio, score_ratio),
        (match_partial, score_partial),
        (match_token_sort, score_token_sort)
    ]
    
    # Select the best match based on the highest score
    best_match, highest_score = max(combined_scores, key=lambda x: x[1])
    
    if highest_score >= 90:  # Adjust the threshold for stricter matching
        return best_match
    else:
        return 'not found'

# Create 'FINAL COURSE' column
final_data['FINAL COURSE'] = final_data['PREFERRED COURSE'].apply(find_best_match)


In [31]:
final_data

,MATCHED CAMPUS,PREFERRED COURSE,FINAL COURSE
0,PHINMA St. Jude College - Quezon City,BS Criminology,BS Criminology
1,PHINMA St. Jude College - Quezon City,BS Accountancy,BS Accountancy
2,PHINMA St. Jude College - Quezon City,OTHERS,OTHERS
3,PHINMA St. Jude College - Quezon City,BS Hospitality Management,BS Hospitality Management
28,PHINMA St. Jude College - Quezon City,BS Information Technology,BS Information Technology
...,...,...,...
35119,PHINMA St. Jude College - Manila,TVL-ICT,not found
37324,PHINMA St. Jude College - Manila,BSED - Math,BSED - Math
37664,PHINMA St. Jude College - Manila,BSED - Science,BSED - Science
40180,PHINMA St. Jude College - Manila,BSBA - Marketing Management,BSBA - Marketing Management


In [35]:
# Rename columns
final_data.rename(columns={'MATCHED CAMPUS': 'PREFERRED CAMPUS', 'PREFERRED COURSE': 'TO DROP', 'FINAL COURSE': 'PREFERRED COURSE'}, inplace=True)


In [36]:
final_data.to_excel('processed_data.xlsx', index=False)